In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.jobs.sutherlandglobal.com/job-details/associate-manager-coding-quality-auditor-in-hc-sss-jobs-1092896")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.














































































          


       Home


   Refer



  Work At Home



  Enterprise





  Sutherland Life



  Diversity and Inclusion



  Careers



  Locations


   North America


   USA



  Canada



  Jamaica





  Latin America


   Mexico



  Colombia





  APAC


   India



  Philippines



  Malaysia



  China





  Europe


   Bulgaria



  Kosovo



  UK





  Middle East


   Egypt



  UAE







  Contact Us





     Login



  Register


   User





  Dashboard
Client Portal
Signout








  





       






 



       


     Home


   Refer



  Work At Home



  Enterprise





  Sutherland Life



  Diversity and Inclusion



  Careers



  Locations


   North America


   USA



  Canada



  Jamaica





  Latin America


   Mexico



  Colombia





  APAC


   India



  Philippines



  Malaysia



  China





  Europe


   Bulgaria



  Kosovo



  UK





  Middle East


   Egypt


In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[]

In [11]:
type(json_res)

list

In [13]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [15]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m

In [17]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

NameError: name 'collection' is not defined

In [101]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [103]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your Coding Quality Audits with AtliQ's Expertise

Dear Hiring Manager,

I came across the Associate Manager, Coding Quality Auditor role at Sutherland and was impressed by the organization's commitment to maintaining high-quality coding standards. As a Business Development Executive at AtliQ, I'd like to introduce you to our AI & Software Consulting services that can help streamline your coding quality audits and compliance processes.

At AtliQ, we have extensive experience in developing tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can help you leverage technology to enhance your coding quality audits, ensuring compliance with federal, state, and regulatory guidelines.

Our capabilities include:

* Developing custom audit management systems to streamline your coding quality audits
* Implementing machine learning algorithms to identify potential coding errors and anomalies
* Int

NameError: name 'python' is not defined